In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as reader

##  Decoding Noise-Free DTMF Signals

#### I. Goertzel Algorithm

We calculate intermediate coefficients using an iterative approach, at the end we have the real and imaginary components of the N-Point DFT for that specific frequency. We then calculate the energy of the signal at that frequency by summing the squares of the real and imaginary parts. We use this inside a for loop that runs through all the frequencies of interest. Our output is an array of energy values for each of the frequencies of interest.

<img width=30% src="goertzel.png">

In [2]:
def goertzel(x, fs, freqs, Ks, N):
    
    output = []
    for k in Ks:
        cos_w = np.cos(2*np.pi*k/N)
        sin_w = np.sin(2*np.pi*k/N)
        c = 2*cos_w
        
        q1 = 0
        q2 = 0
        for n in range(0, N):
            q0 = x[n] + c*q1 - q2
            q2 = q1
            q1 = q0

        real = cos_w*q1 - q2
        imag = sin_w*q1
        energy = np.sqrt(real**2+imag**2) 
        output.append(energy)

    return np.array(output)

#### II. Create lookup table

A function to find a digit given two frequencies. It uses a matrix consisting of the different frequency combinations and a 1D array of corresponding digits. For each pair of frequencies given, the keypad frequency matrix is traversed and used as comparison. If the frequency values match, the index is used to get the value from the digits array. I use a set so the order of the frequencies is not relevant.

<img width=30% src="touchtone.png">

In [3]:
def findDigit(x):
    rows = [697, 770, 852, 941]
    columns = [1209, 1336, 1477, 1633]

    freqs = []
    for r in rows:
        for c in columns:
            freqs.append([c, r])
            
    freqs = np.array(freqs)
    touchtone = np.array(["1", "2", "3", "A", "4", "5", "6", "B", "7", "8", "9", "C", "*", "0", "#", "D"])
    
    for i, f in enumerate(freqs):
        if set(x)==set(f): return touchtone[i]
    return "Invalid"

#### III. Get energy

To get the energy at the frequencies of interest, we first compute the Ks and then call the goertzel algorithm previously defined. To compute the Ks we multiply each frequency by the number of samples N and divide by the sampling frequency. For this case we use a N of 205 and a fs of 8000Hz

In [4]:
def getEnergy(x, fs, freqs, N):
    #Calculate Ks for frequencies
    Ks = np.round(freqs * N / fs)
    #Get energy at each frequency of interest
    y = goertzel(x, fs, freqs, Ks, N)
    return y

#### IV. Digit decoder

The dtmfdecode function takes the signal and the sampling frequency as input and outputs a matching digit on the touchtone keypad. To decode the signal we first compute the energy of the signal given the frequencies of interest. (Th frequencies used in the touchtone keypad). Then we make a decision if the tone is present or not based on a threshold. I decided to set my threshold as the maximum value divided by 2. I then filter the tones based on this threshold. If there are exactly two tones (frequencies) we can use the findDigit function to search for the digit in the lookup table. If there are more or less, it means it is not a dtmf digit.

In [5]:
def dtmfdecode(x, fs):
    #Number of samples
    N = 205
        
    #Frequencies of interest
    freqs = np.array([697, 770, 852, 941, 1209, 1336, 1477, 1633])  
    freqs_energy = getEnergy(x, fs, freqs, N)
    
    #Find if tone is present based on energy threshold
    tone_present = freqs_energy > np.amax(freqs_energy)/2
    
    #Filter present tones
    tones = freqs[tone_present]

    #At this point we must have two tones to find digit
    if len(tones) != 2: return "Not found"
    
    #Find digit based on tones
    return findDigit(tones)

#### V. Test implementation

To test the implementation, we import all soundfiles, extract signal and sampling frequency and run dtmfdecode(). We observe that they are correctly decoded.

In [6]:
fs, touchtone_0 = reader.read('./soundfiles/touchtone_0.wav')
_ , touchtone_1 = reader.read('./soundfiles/touchtone_1.wav')
_ , touchtone_2 = reader.read('./soundfiles/touchtone_2.wav')
_ , touchtone_3 = reader.read('./soundfiles/touchtone_3.wav')
_ , touchtone_4 = reader.read('./soundfiles/touchtone_4.wav')
_ , touchtone_5 = reader.read('./soundfiles/touchtone_5.wav')
_ , touchtone_6 = reader.read('./soundfiles/touchtone_6.wav')
_ , touchtone_7 = reader.read('./soundfiles/touchtone_7.wav')
_ , touchtone_8 = reader.read('./soundfiles/touchtone_8.wav')
_ , touchtone_9 = reader.read('./soundfiles/touchtone_9.wav')
_ , touchtone_pound = reader.read('./soundfiles/touchtone_pound.wav')
_ , touchtone_star = reader.read('./soundfiles/touchtone_star.wav')

In [7]:
print(dtmfdecode(touchtone_0, fs))
print(dtmfdecode(touchtone_1, fs))
print(dtmfdecode(touchtone_2, fs))
print(dtmfdecode(touchtone_3, fs))
print(dtmfdecode(touchtone_4, fs))
print(dtmfdecode(touchtone_5, fs))
print(dtmfdecode(touchtone_6, fs))
print(dtmfdecode(touchtone_7, fs))
print(dtmfdecode(touchtone_8, fs))
print(dtmfdecode(touchtone_9, fs))
print(dtmfdecode(touchtone_pound, fs))
print(dtmfdecode(touchtone_star, fs))

0
1
2
3
4
5
6
7
8
9
#
*


## 2. Decoding Noise-Free DTMF Signals

For the robust decoder, we first perform the same steps as the dtmfdecode function to find the tones present in the signal. However, we need to perform an additional step to make sure the energy of the signal is low for the 2nd harmonics of these tones. 
We first compute the 2nd harmonics by multiplying by 2 the identified tones. We get the energy of these harmonics using the getEnergy function that uses Goertzel algorithm. We then compute a energy ratio by dividing the energy of the original tones (fundamental frequencies) by the energy of their second harmonics. If this ratio is big, it means the energy of the fundamental frequencies is large compared to the harmonics, and therefore it is a valid dtmf tone. However, if the ratio is small (approaches 1) it means the energy of the harmonics is large and therefore the tone was probably produced by other source so it should not be decoded. 
To make the decision, we use a threshold on the energy ratio, I decided to use "3" as the threshold ratio. This means that if the energy of the harmonics is at least 1/3 of the energy of the fundamental frequency, the signal should not be considered as a dtmf tone.
I filter out tones with harmonics using this threshold and finally perform digit decoding for the leftover tones using the findDigit function.

In [8]:
def dtmfrobustdecode(x, fs):
    #Number of samples
    N = 205
        
    #Frequencies of interest
    freqs = np.array([697, 770, 852, 941, 1209, 1336, 1477, 1633])  
    freqs_energy = getEnergy(x, fs, freqs, N)
    
    #Find if tone is present based on energy threshold
    tone_present = freqs_energy > np.amax(freqs_energy)/3
    
    #Filter present tones
    tones = freqs[tone_present]

    #Find energy for present tones
    tones_energy = freqs_energy[tone_present]
        
    #Find 2 harmonics of present tones
    harmonics = 2 * tones
    #Find energy for these harmonics
    harmonics_energy = getEnergy(x, fs, harmonics, N)
    
    #Compute energy ratio
    energy_ratio = tones_energy/harmonics_energy

    #Find if harmonics based on threshold
    harmonic_present = energy_ratio < 3
    
    #Filter out tones with harmonics
    pure_tones = tones[harmonic_present == False]

    #At this point we must have two tones to find digit
    if len(pure_tones) != 2: return "Not found"
    
    #Find digit based on tones
    return findDigit(pure_tones)

### Test Robust system

To test the robust system I synthesized signals and added 2nd harmonics.

In [9]:
def SineSignal(freq, amp=1, end_time=2, Fs = 8000):
    t = np.linspace(0, end_time, int(end_time*Fs))
    y = amp*np.sin(2*np.pi*freq*t)
    return y, t

#### Test on simple tones
Using simple tones with only fundamental frequencies, the signal is correctly identified as a dtmf tone.

In [10]:
f_1 = 1336
f_2 = 941
amp = 1000
y1, t = SineSignal(f_1, amp, 970, fs)
y2, t = SineSignal(f_2, amp, 970, fs)
y = y1+y2
dtmfrobustdecode(y, fs)

'0'

#### Test by adding 2 harmonics
However, whe we add 2nd harmonics, the system outputs "Not found" meaning that it correctly neglected tones that have harmonics.

In [11]:
h1, t = SineSignal(2*f_1, 0.5*amp, 970, fs)
h2, t = SineSignal(2*f_2, 0.5*amp, 970, fs)
y = y1+y2+h1+h2
dtmfrobustdecode(y, fs)

'Not found'

#### Test on sample
We tested the robust decoded on one of the original audio samples and got a correct decoding.

In [12]:
dtmfrobustdecode(touchtone_0, fs)

'0'

#### Decode file

In [13]:
fs, mystery = reader.read('./soundfiles/mystery.wav')

In [14]:
def decodeMultitone(x, N):
    chunk_size = round(len(x)/N)
    number = []
    for i in range(0,N):
        start = i*chunk_size
        end = start + chunk_size
        chunk = x[start:end]
        digit = dtmfrobustdecode(chunk, fs)
        print(digit)
        if digit != "Not found": number.append(digit)
    return number

In [15]:
decodeMultitone(mystery, 9)

Not found
8
6
7
Not found
5
3
0
9


['8', '6', '7', '5', '3', '0', '9']